In [5]:
import requests
import json
import numpy as np
import pandas as pd
from copy import deepcopy
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils') 
from Preprocessing import Preprocessor
from constants import RUSSIAN_TIME_ZONES
from Preprocessing import vincenty_dist

mypath = '/mnt/HARD/MinMax94/data/data_all/CSV/Raw_extended/'

## Station_config

In [6]:
stations_mm94 = pd.read_csv('/mnt/HARD/MinMax94/data/data_all/CSV/stations_mm94_def.csv')
stations_rp5 = pd.read_csv('/mnt/HARD/MinMax94/data/data_all/CSV/stations_rp5_def.csv')

In [7]:
stations = deepcopy(stations_mm94)
stations['station_type'] = "road"
stations['station_nature'] = "real"
stations['timezone'] = stations['timezone'].apply(lambda row: RUSSIAN_TIME_ZONES[row])
stations['road_category'] = 1
#stations['roadlayers'] = str({"1": {"type": "asphalt", "thickness": 0.2}})
stations = stations[['station_id', 'timezone', 'longitude', 'latitude', 
              'station_type', 'station_nature', 'road_category']]

In [8]:
mm94_station_id = 113
exact_station = stations[stations['station_id']==mm94_station_id]
station_config = exact_station.to_dict(orient='records')[0]
station_config['roadlayers'] = {"1": {"type": "asphalt", "thickness": 0.2}}
#with open("file", "w") as file:
#    json.dump(exact_station_data, file)
    
#with open("file", "r") as file:
#    station_config = json.load(file)

station_config

{'latitude': 53.603,
 'longitude': 54.446999999999996,
 'road_category': 1,
 'roadlayers': {'1': {'thickness': 0.2, 'type': 'asphalt'}},
 'station_id': 113,
 'station_nature': 'real',
 'station_type': 'road',
 'timezone': 'MSK'}

## Road_config

In [9]:
road_config = {
    "maintainability_level": "low",
    "category": "1А"}

## Global_forecast

In [10]:
## find nearest wmo station

station_data = stations_mm94[stations_mm94['station_id']==mm94_station_id]
mm94_coords = station_data[['latitude', 'longitude']].values[0]

nearest_rp5_station_id = stations_rp5.loc[[stations_rp5.apply(
                                lambda x: vincenty_dist(mm94_coords, (x['latitude'], x['longitude'])), 
                                axis=1).idxmin()]]['station_id'].values[0]

In [11]:
wmo_station_id = nearest_rp5_station_id
mypath = '/mnt/HARD/MinMax94/data/data_all/CSV/RP5/'
preprocessor = Preprocessor()

rp5 = pd.read_csv(mypath + str(wmo_station_id) + '.csv', sep=';', skiprows=6, index_col=False,
                 dtype={'VV': str, 'RRR': str})
date_time_col = [col for col in rp5.columns if col.startswith('Местное время')][0]
rp5 = rp5.rename(columns={date_time_col: 'Местное время'})
rp5['station_id'] = wmo_station_id

mmx_wmo_data = preprocessor.ConvertData(rp5, from_format="RP5", to_format="Mmx")
del mmx_wmo_data['data_p_weather'], mmx_wmo_data['data_precip_interval']
mmx_wmo_data = preprocessor.AddUTC(mmx_wmo_data, '/mnt/HARD/MinMax94/data/data_all/CSV/stations_rp5_def.csv')
mmx_wmo_interpolated = preprocessor.InterpolatePatterns(mmx_wmo_data)

forecast_data = preprocessor.ConvertData(mmx_wmo_interpolated, from_format="Mmx", to_format="Metro")
forecast_data = forecast_data.set_index('date_time_utc')
forecast_data["p_weather"] = np.nan
del forecast_data['station_id'], forecast_data['wind_gusts']
forecast_data = forecast_data.fillna(9999)

In [12]:
current_date_time = pd.Timestamp(2015, 12, 30, 13, 0)
predict_future_delta = pd.Timedelta(45, unit='h')
predict_previous_delta = pd.Timedelta(3, unit='h')
global_forecast = forecast_data[(forecast_data.index >= pd.Timestamp(2015, 12, 30, 10, 0)) & \
                                (forecast_data.index <= pd.Timestamp(2016, 1, 1, 14, 0))]
global_forecast = global_forecast.set_index('date_time_metro', drop=True)
global_forecast_json = global_forecast.to_dict(orient='index')

## RWIS data

In [13]:
mypath = '/mnt/HARD/MinMax94/data/data_all/CSV/Raw_extended/'

# reading loaded csv files from data_csv directory, output is a list (length=number of stations) of raw df
raw = pd.read_csv(mypath + str(mm94_station_id) + '_raw.csv', parse_dates = ['date_time']) 
raw = raw.reset_index(drop=True)

raw_data = preprocessor.SelectFeatures(raw)
raw_data = preprocessor.PivotTable(raw_data)
mmx_data = preprocessor.ConvertData(raw_data, from_format="Raw", to_format="Mmx")
mmx_data = preprocessor.AddUTC(mmx_data)
mmx_patterns = preprocessor.CreatePatternList(mmx_data)
mmx_interpolated = preprocessor.InterpolatePatterns(mmx_patterns)

rwis_df = preprocessor.ConvertData(mmx_interpolated, from_format="Mmx", to_format="Metro")
rwis_df = rwis_df.set_index('date_time_utc')
rwis_df['salinity'] = 0
rwis_df['freezing_point'] = 0
del rwis_df['cloudiness'], rwis_df['station_id']
rwis_df = rwis_df.fillna(9999)

In [14]:
rwis_previous_delta = pd.Timedelta(12, unit='h')
rwis_data = rwis_df[(rwis_df.index >= pd.Timestamp(2015, 12, 30, 7, 0)) & \
                                (rwis_df.index <= pd.Timestamp(2015, 12, 30, 13, 0))]
rwis_data = rwis_data.set_index('date_time_metro', drop=True)
rwis_data_json = rwis_data.to_dict(orient='index')
rwis_data_json

{'2015-12-30 07:00 UTC': {'freezing_point': 0.0,
  'humidity': 98.150000000000006,
  'precipitation_intensity': 0.20000000000000001,
  'precipitation_type': 20.0,
  'pressure': 746.0,
  'salinity': 0.0,
  't_air': -14.550000000000001,
  't_dew_point': -14.800000000000001,
  't_road': -9.25,
  't_underroad': -8.9000000000000004,
  'wind_direction': 112.5,
  'wind_gusts': 2.0499999999999998,
  'wind_speed': 1.25},
 '2015-12-30 07:30 UTC': {'freezing_point': 0.0,
  'humidity': 98.299999999999997,
  'precipitation_intensity': 0.60000000000000009,
  'precipitation_type': 20.0,
  'pressure': 746.0,
  'salinity': 0.0,
  't_air': -14.5,
  't_dew_point': -14.699999999999999,
  't_road': -8.9499999999999993,
  't_underroad': -8.8000000000000007,
  'wind_direction': 112.5,
  'wind_gusts': 2.0,
  'wind_speed': 1.3},
 '2015-12-30 08:00 UTC': {'freezing_point': 0.0,
  'humidity': 97.650000000000006,
  'precipitation_intensity': 0.69999999999999996,
  'precipitation_type': 20.0,
  'pressure': 746.0,


## All together

In [17]:
import simplejson
test_path = '/home/ndsviriden/metro_test.txt'

data_for_metro = {"station_config": station_config, 
                  "road_config": road_config,
                  "global_forecast": global_forecast_json,
                  "rwis_data": rwis_data_json}

In [18]:
with open(test_path, "w") as file:
    simplejson.dump(data_for_metro, file, indent=4, ignore_nan=True)
    
with open(test_path, "r") as file:
    data_for_metro = json.load(file)